# Init 

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import warnings
from os import path, listdir
from IPython.display import Video

from utils import *
from pre_processing import *
from change_detection import *
from object_classification import *
from post_processing import *

warnings.filterwarnings('ignore') 

# Load Video

In [ ]:
video_path = path.join('data', 'videos', 'approved2.mp4')
colored_vid = load_video(video_path=video_path, gray_scale=False)
Video(video_path, height=400, width=600)

# Preprocessing

## Histograms Calculation

### Preprocessed images outline

In [ ]:
images_path = path.join('data', 'object_outline')
image_list = load_images(images_path)

### Calculating the outline histograms

In [ ]:
window_size = 15
histograms_mean_dict = {"RED":[], "GREEN":[], "BLUE":[]}
for img_num, img in enumerate(image_list):
    r_histograms_mean_list = generate_patches_histograms_mean(img[:,:,0], window_size)
    g_histograms_mean_list = generate_patches_histograms_mean(img[:,:,1], window_size)
    b_histograms_mean_list = generate_patches_histograms_mean(img[:,:,2], window_size)
    histograms_mean_dict["RED"].append(r_histograms_mean_list)
    histograms_mean_dict["GREEN"].append(g_histograms_mean_list)
    histograms_mean_dict["BLUE"].append(b_histograms_mean_list)

### Clustering the hitograms mean and calculating the mean for each cluster

In [ ]:
bins = 2
allowed_histogram_means = calculate_allowed_histograms_means(histograms_mean_dict, bins)

In [ ]:
allowed_histogram_means

## Object Size calculation

### Creating a grid of the input image

In [ ]:
images_path = path.join('data', 'preprocessing_images')
image_to_grid = load_images(images_path)[0] # taking the first image based on the knowledge all images sizes are the same

In [ ]:
grid_size = 5 # grid the image with boxes of 20X20
image_x_grid = np.arange(0, image_to_grid.shape[1], grid_size)
image_y_grid = np.arange(0, image_to_grid.shape[0], grid_size)

### Creating a database for approved object size information

In [ ]:
first_object_location = [100, 325, 260, 360]
object_size_info = get_grids(image_x_grid, image_y_grid, first_object_location)

In [ ]:
object_size_info

In [ ]:
first_approved_object_info = {"RED": allowed_histogram_means["RED"],
                             "GREEN": allowed_histogram_means["GREEN"],
                             "BLUE": allowed_histogram_means["BLUE"],
                             "location": [object_size_info]}

In [ ]:
first_approved_object_info

# Change Detection 

## Set Parameters 

In [ ]:
univariate = True
cut_percent = 100
predict = True
k=3
t=0.7
alpha=2.5
learning_rate=0.5
k_warm_up=1
model_path = path.join('data', 'models', f'{univariate}-{cut_percent}.pickle')

colored_vid = resize(colored_vid, percent=cut_percent, gray_scale=False)

if univariate:
    full_vid = load_video(video_path=video_path, gray_scale=True)
    vid = resize(full_vid, percent=cut_percent, gray_scale=True)
else:
    vid = colored_vid.copy()

## GMM

### Create Foreground Mask 

In [ ]:
v_fg_mask = change_detection(vid, k=k, t=t, alpha=alpha, learning_rate=learning_rate, 
                             k_warm_up=k_warm_up, univariate=univariate, model_path=model_path, predict=predict)

In [ ]:
play_video_by_images(v_fg_mask, frame_rate=10)

### Postprocessing - Improve foreground mask

In [ ]:
v_fg_mask_pp = improve_foreground(v_fg_mask)

In [ ]:
play_video_by_images(v_fg_mask_pp, frame_rate=10)

### Find Contours

In [ ]:
frames_contours = find_contours(v_fg_mask_pp.copy())
frames_contours

# Object Classification 

In [ ]:
for frame_index, detected_objects_location in frames_contours.items():
    frame = colored_vid[frame_index]
    num_of_objects = len(detected_objects_location)
    objects_info = get_detected_objects_info(frame, detected_objects_location, image_x_grid, image_y_grid, window_size)
    approved_objects = match_with_approved_objects(num_of_objects, objects_info, first_approved_object_info)
    save_images_with_boxes(frame, frame_index, approved_objects, detected_objects_location, out_dir = path.join('data', 'object_classification'))

## Decide on Alert or Not 

# Measures 